Scikit-learn harjoitus 2
======================

Tämän harjoituksen tarkoitus on opettaa hieman edistyksellisimpiä toimintoja.

Asentaaksesi tarvittavat paketit omalla koneellasi harjoituksen suorittamista varten:
```
$ pip3 install scikit-learn pandas imbalanced-learn matplotlib==2.2.2
```



## Kuvien käyttäminen datana
Aloita lataamalla harjoitusdata. Tällä kertaa käytetään [MNIST](https://en.wikipedia.org/wiki/MNIST_database)-aineistoa,
joka sisältää käsinkirjoitettuja numeroita 0-9 8x8 pikselin kuvina.

In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
dataset = datasets.load_digits()

# Näytetään ensimmäinen kuva
plt.imshow(dataset.images[0], cmap=plt.cm.gray_r, interpolation='nearest')
plt.title('Esimerkkikuva')
plt.show()

print("Esimerkkikuva pikseliarvoina: \n", dataset.images[0])

# dataset-olio tarjoaa havainnot myös valmiiksi prosessoituna yhdeksi vektoriksi/listaksi
print("Esimerkkikuva havaintovektorina: \n", dataset.data[0])


Jotta kuvista, eli n*m-taulukoista päästään yksiulotteiseen havaintovektorimuotoon, ne pitää prosessoida.

In [2]:
n_samples = len(dataset.images)
data = dataset.images.reshape((n_samples, -1))
if np.array_equal(data, dataset.data):
    print("Kuvat ovat nyt yksiulotteisia")

Nyt edellistä harjoitusta hyväksi käyttäen suorita vaiheet ja luokittele malli haluamallasi algoritmilla 
ja parametreilla. Yritä päästä yleiseen tarkkuuteen OA>=0.9.

Jaa luomasi X ja luokat y opetus- ja testijoukkoihin `X_train, X_test, y_train ja  y_test` siten, 
että testijoukon osuus on 20% ja opetusjoukon osuus on 80% havainnoista. 
Käytä jaossa `random_state`-parametrin arvona 42

In [3]:
from sklearn.model_selection import train_test_split
X = data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # TODO: None

## Esikäsittely
Suorita standardointi.

In [4]:
# Evaluoi
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix

## Epätasapainon korjaaminen
Jos luokkien määrä havaintojen välillä on epätasapainossa, malli suosii helposti luokkia, joissa on enemmän havaintoja, kuin toisissa. Tämä on yksi koneoppimisen suurista ongelmista. Yksi keino hallita ongelmaa, on havaintojen resamplaus, eli havaintojen määrän muuttaminen opetusjoukossa siten, että luokkien välinen epätasapaino pienenisi.

Tähän on kaksi menetelmää: 
* Oversample: Havaintojen lisääminen pienimpiin luokkiin
* Undersample: Havaintojen poistaminen suurimmista luokista


Molempiin menetelmiin on monta algoritmiä mainiossa [Imbalanced-learn](https://imbalanced-learn.readthedocs.io/en/stable/index.html)-kirjastossa.

Luodaan aluksi epätasapainoinen opetusaineisto alkuperäisestä opetusaineistosta.

In [ ]:
from imblearn.datasets import make_imbalance
from collections import Counter, OrderedDict

sampling_strategy = {0: 50, 1: 154, 2: 144, 3: 70, 4: 135, 5: 135, 6: 80, 7: 145, 8: 144, 9: 60}
X_train_imb, y_train_imb =  make_imbalance(X_train, y_train, sampling_strategy=sampling_strategy)

print("Alkuperäinen jakauma", dict(OrderedDict(sorted(Counter(y_train).items(), key=lambda t: t[0]))))
print("Epätasapainoinen jakauma", dict(OrderedDict(sorted(Counter(y_train_imb).items(), key=lambda t: t[0]))))

Suorita yksinkertainen luokittelu molemmilla aineistoilla haluamallasi luokittelualgoritmilla.

In [17]:
from sklearn.svm import SVC
clf = SVC()

clf.fit(X_train, y_train)
print("Alkuperäinen OA: {:.2f}".format(clf.score(X_test, y_test)))

clf.fit(X_train_imb, y_train_imb)
print("Epätasapainoisen OA: {:.2f}".format(clf.score(X_test, y_test)))

Epätasapainon korjaamiseksi kokeillaan nyt molempia resample-menetelmiä. Käytä yksinkertaisia satunnaisuuteen perustuvia [RandomOverSampler](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html#imblearn.over_sampling.RandomOverSampler)- ja [RandomUnderSampler](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler)-luokkia.

In [19]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


ros = RandomOverSampler()
X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train_imb, y_train_imb)
rus = RandomUnderSampler()
X_train_undersampled, y_train_underrsampled = rus.fit_resample(X_train_imb, y_train_imb)

print("Jakauma oversamplen jälkeen", 
      dict(OrderedDict(sorted(Counter(y_train_oversampled).items(), key=lambda t: t[0]))))
print("Jakauma undersamplen jälkeen",
      dict(OrderedDict(sorted(Counter(y_train_underrsampled).items(), key=lambda t: t[0]))))

clf.fit(X_train_oversampled, y_train_oversampled)
print("Oversampled OA: {:.2f}".format(clf.score(X_test, y_test)))

clf.fit(X_train_undersampled, y_train_underrsampled)
print("Undersampled OA: {:.2f}".format(clf.score(X_test, y_test)))

## Ristivalidointi
Myös testijoukon ylisovitus on mahdollista kun mallia säädetään paljon. Tällöin on mahdollista käyttää kolmatta validointijoukkoa, mutta ristivalidointi on toinen (ja monesti parempi) vaihtoehto. Siinä käytetään hyväksi vain opetusjoukkoa, joka jaetaan osiin siten, että vuorollaan jokainen havainto on ristivalidoinnin sisäisessä opetusjoukossa ja vuorollaan testijoukossa. Näitä kutsutaan jaoiksi (engl. split). Tällöin oikeaa testijoukkoa voi säästää aivan viimeiseen säädetyn mallin validointiin saakka.

Suorita alla yksinkertainen ristivalidointi käyttäen [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)-metodia ja haluamaasi luokittelualgoritmia. Aseta `CV`-parametri arvoksi 5 ja `n_jobs`-parametri arvoon -1.

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
clf = SVC(gamma=0.01)
scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

`n_jobs`-parametri mahdollisti rinnakkaisen laskennan eri mallien välille. Tämä parametri löytyy jokaisesta ristivalidointia käyttävästä metodista ja myös joistain luokittelijoista, kuten Random Forest. Sitä suositellaan käytettäväksi aina kun on mahdollista, mutta harjoituksissa sen käyttöä on vältetty suorituskyvyn vuoksi.

### Putkittaminen
Ristivalidoinnissa datan esikäsittely, joka riippuu datasta, on hyvä suorittaa erikseen jokaiselle jaolle. Tämä on helpoin tehdä käyttämällä putkitusta ([pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline)). Sillä tavoin on mahdollista kytkeä paljon esikäsittelyvaiheita yhteen ja suorittaa vaiheet jokaisen jaon kohdalla. Putkitusta on syytä käyttää myös hyperparametrioptimoinnin kanssa.

Suorita ensin äskeinen ristivalidointi uudestaan käyttäen moduulia [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) vaiheita [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) ja [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), sekä haluaamasi luokittelualgoritmia. 


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

pipe = Pipeline([("standard_scaler", StandardScaler()), ("pca", PCA()), ("clf", SVC(gamma=0.01))])
scores = cross_val_score(pipe, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

Suorita sama uudestaan käyttäen helppokäyttöisempää metodia [make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline)

In [20]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), PCA(), SVC(gamma=0.01))
scores = cross_val_score(pipe, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

Lue halutessasi lisää ristivalidoinnista [täältä](https://scikit-learn.org/stable/modules/cross_validation.html).